In [249]:
import pandas as pd
import numpy as np
from random import randint
from scipy.stats import multivariate_normal
from tqdm import tqdm_notebook as tqdm      # for tracking loops
from sklearn.metrics import normalized_mutual_info_score as nmi_score
import operator as op
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
train = pd.read_csv("./compound.csv",names = ["x1", "x2", "label"])
train.head()

,x1,x2,label
0,26.75,22.15,1
1,29.80,22.15,1
2,31.55,21.10,1
3,27.70,20.85,1
4,29.90,19.95,1


In [3]:
train_data = train.drop("label",axis=1)
train_data = train_data.values
train_labels = train["label"].values

dim = 2

# test_data = test.drop("label",axis=1)
# test_data = test_data.values
# test_labels = test["label"].values

print("Labels: ",train_labels)
tr_data_labels = np.c_[train_data,train_labels]
np.random.shuffle(tr_data_labels)
tr_data_labels

train_labels = tr_data_labels[:,dim]
train_data = np.delete(tr_data_labels, dim, 1)
print("train_data.shape: ", train_data.shape)
print("train_labels.shape: ", train_labels.shape)
# print(test_data.shape)
# print(test_labels.shape)
# print(train_labels)

Labels:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
train_data.shape:  (399, 2)
train_labels.shape:  (399,)


# EM Algorithm

## Mixture of 5 Gaussians

In [65]:
# can make a class.


### E-step

In [142]:
def E_step(train_data, pi_s, u_s, sigma_s, N=60000, K=10, d=784):
    q_values = [[0 for j in range(N)] for i in range(K)]
    sums = [0 for i in range(N)]
    
    for i in range(K):
        for j in range(N):
            pdf = multivariate_normal.pdf(train_data[j], mean=u_s[i], cov=sigma_s[i],allow_singular=True)
            q_values[i][j] = pi_s[i]*pdf
            sums[j] += q_values[i][j]
    
    q_values = np.array(q_values)
    for i in range(K):
        for j in range(N):
            q_values[i][j] = q_values[i][j] / sums[j]
    q_values = np.array(q_values)
    return q_values

### M-step

In [143]:
def M_step(train_data, q_values, pi_s, u_s, sigma_s, N=60000, K=10, d=784):
    pi_s = np.sum(q_values,axis=1)
    for i in range(K):
        tmp_mean = 0
        for j in range(N):
            tmp_mean += q_values[i][j]*train_data[j]
        u_s[i] = (1/pi_s[i])*tmp_mean
        
    for i in range(K):
        tmp_sig = np.zeros((d,d))
        for j in range(N):
            mat = np.matrix(train_data[j]-u_s[i])
            tmp_sig += q_values[i][j]*np.matmul(mat.T, mat)
        sigma_s[i] = (1/pi_s[i])*tmp_sig
    
    pi_s = pi_s / N
    
    return pi_s, u_s, sigma_s

In [344]:
N = 0
K = 5
d = 2
train_data_arranged = [[] for i in range(7)]

## Adding random small elements to sigma matrices

In [345]:
def nonSingularize(A,n):
    for i in range(n):
        for j in range(n):
            A[i][j] += np.random.uniform(0,0.01)
    return A

In [144]:
def calc_logLikelihood(train_data_arranged, u_s, sigma_s, pi_s, K_classes = 5):
    log_lilelihood_new = 0
    for i in range(len(train_data_arranged)):
            tmp = 0
            for j in range(K_classes):
                pdf = multivariate_normal.pdf(train_data_arranged[i],mean=u_s[j], cov=sigma_s[j],allow_singular=True)
                tmp += pi_s[j]*pdf
            tmp = np.log(tmp)
            log_lilelihood_new += tmp
    return log_lilelihood_new

## E-M for Compound Dataset

In [244]:
K_classes = 6

d = 2
N_points = len(train_data)
pi_s= [1/K_classes for i in range(K_classes)]

# Because data is not online, we can find the max and min values for each dimension
# and use them to initialize means, though a rough value used.
u_s = [np.random.randint(5,30,d) for i in range(K_classes)]

# Could have inilialized using data, but wanted to see the results with
# the initializations as identity matrices
sigma_s = [np.eye(d) for i in range(K_classes)]

eps = 0.009

log_lilelihood_old = 0
log_lilelihood_new = 1000   # some random large value
u_s,sigma_s

([array([22,  8]),
  array([10, 12]),
  array([28, 16]),
  array([29, 19]),
  array([ 7, 27]),
  array([27, 22])],
 [array([[ 1.,  0.],
         [ 0.,  1.]]), array([[ 1.,  0.],
         [ 0.,  1.]]), array([[ 1.,  0.],
         [ 0.,  1.]]), array([[ 1.,  0.],
         [ 0.,  1.]]), array([[ 1.,  0.],
         [ 0.,  1.]]), array([[ 1.,  0.],
         [ 0.,  1.]])])

In [245]:
log_lilelihood_new = calc_logLikelihood(K_classes=K_classes, sigma_s=sigma_s, train_data_arranged=train_data, u_s=u_s, pi_s=pi_s)
train_data_arranged = train_data
np.absolute(log_lilelihood_new-log_lilelihood_old)

9420.5803442342985

In [246]:
pbar = tqdm(total = 200+1)
step = 0
while np.absolute(log_lilelihood_new - log_lilelihood_old) > eps :
    log_lilelihood_old  = log_lilelihood_new
    q_values            =  E_step(train_data_arranged, pi_s, u_s, sigma_s, N=N_points, K=K_classes, d=2)
    pi_s, u_s, sigma_s  =  M_step(train_data_arranged, q_values, pi_s, u_s, sigma_s, N=N_points, K=K_classes, d=2)
    
    log_lilelihood_new = calc_logLikelihood(train_data_arranged, u_s, sigma_s, pi_s, K_classes = K_classes)

    step += 1
    print("After %d iter :" %(step))
    print("\tlog_lilelihood_old: %f" %(log_lilelihood_old))
    print("\tlog_lilelihood_new: %f" %(log_lilelihood_new))
    print("\t Diff: %f\n" %(log_lilelihood_new - log_lilelihood_old))
    pbar.update(1)

After 1 iter :
	log_lilelihood_old: -9420.580344
	log_lilelihood_new: -2426.640151
	 Diff: 6993.940193

After 2 iter :
	log_lilelihood_old: -2426.640151
	log_lilelihood_new: -2397.830350
	 Diff: 28.809802

After 3 iter :
	log_lilelihood_old: -2397.830350
	log_lilelihood_new: -2380.938587
	 Diff: 16.891763

After 4 iter :
	log_lilelihood_old: -2380.938587
	log_lilelihood_new: -2370.808659
	 Diff: 10.129928

After 5 iter :
	log_lilelihood_old: -2370.808659
	log_lilelihood_new: -2363.579702
	 Diff: 7.228957

After 6 iter :
	log_lilelihood_old: -2363.579702
	log_lilelihood_new: -2357.854277
	 Diff: 5.725425

After 7 iter :
	log_lilelihood_old: -2357.854277
	log_lilelihood_new: -2353.558194
	 Diff: 4.296083

After 8 iter :
	log_lilelihood_old: -2353.558194
	log_lilelihood_new: -2350.667476
	 Diff: 2.890719

After 9 iter :
	log_lilelihood_old: -2350.667476
	log_lilelihood_new: -2348.765022
	 Diff: 1.902453

After 10 iter :
	log_lilelihood_old: -2348.765022
	log_lilelihood_new: -2347.449431
	

In [247]:
log_lilelihood_new - log_lilelihood_old

0.0080733872432574572

In [250]:
q_values            =  E_step(train_data_arranged, pi_s, u_s, sigma_s, N=N_points, K=K_classes, d=2)

predictions = np.argmax(q_values, axis=0)

val = [0 for i in range(K_classes)]
diams = [0 for i in range(K_classes)]

for i in range(N_points):
        val[int(predictions[i])] += np.power(np.linalg.norm(train_data[i]-u_s[predictions[i]]),2)
        for j in range(N_points):
            if predictions[i] == predictions[j]:
                dmtr = np.linalg.norm(train_data[i]-train_data[j])
                if dmtr > diams[predictions[i]]:
                    diams[predictions[i]] = dmtr


print("\nsq_dists: \n",val)
print("total sq_dists: ",sum(val))
print("\ndiameters: ",diams)
print("\nnmi score : ", nmi_score(predictions,train_labels))
print("\n(covariances init. as Identity matrices)")
print("(means are init. as ramdom values from range (min_data,max_data) for each dimension (ROUGHLY))")


sq_dists: 
 [1278.1004475728646, 318.15942214211549, 2571.8201834791084, 554.90367565529561, 251.25690059894168, 264.85299394697165]
total sq_dists:  5239.0936234

diameters:  [11.297123527695001, 8.0687669442114878, 20.796634343085422, 8.6130714614474204, 7.8517513969814416, 7.6526139325069824]

nmi score :  0.832543462711

(covariances init. as Identity matrices)
(means are init. as ramdom values from range (min_data,max_data) for each dimension (ROUGHLY))
